In [1]:
# import libraries
import requests
import urllib
from bs4 import BeautifulSoup
import simplejson as json
from urllib.request import Request, urlopen
from fake_useragent import UserAgent
from selenium import webdriver
import time
import re
import pandas as pd
import json
import datetime
import numpy as np
import unicodedata
from collections import Counter
from nltk import word_tokenize, sent_tokenize
import matplotlib.pyplot as plt
import pickle
from   sklearn.feature_extraction.text import TfidfVectorizer
from   sklearn.feature_selection import SelectKBest, mutual_info_classif
from   sklearn.linear_model import LogisticRegression, LinearRegression
from   sklearn.model_selection import cross_val_score
from   sklearn.preprocessing import StandardScaler
import spacy
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from   sklearn.cluster import KMeans, SpectralClustering, DBSCAN, OPTICS, AgglomerativeClustering
import os
import string
import copy
from   collections import defaultdict
from   nltk.corpus import stopwords
from   sklearn.feature_selection import SelectKBest, mutual_info_regression
from wordcloud import WordCloud 

In [2]:
test = {'cik': ['123'], 'symbol': ['test'], 'company': ['apple'], 'fiscal_year': '2018', 'risk': 'abandon', 'url': 'none', 'price': 100}

In [3]:
df = pd.DataFrame(test)
len(df)

1

In [4]:
df.head()

,cik,symbol,company,fiscal_year,risk,url,price
0,123,test,apple,2018,abandon,none,100


In [5]:
# Making stopwords list
stoplist = stopwords.words('english')
for el in [i for i in string.punctuation]:
    stoplist.append(el)

In [6]:
emolex_file = os.path.join('emolex.txt')

In [7]:
# read_emolex function from INFO 3350 problem set code

emolex_file = os.path.join('emolex.txt')

def read_emolex(filepath=None):
    '''
    Takes a file path to the emolex lexicon file.
    Returns a dictionary of emolex sentiment values.
    '''
    if filepath==None: # Try to find the emolex file
        filepath = os.path.join('emolex.txt')
        if os.path.isfile(filepath):
            pass
        elif os.path.isfile('emolex.txt'):
            filepath = 'emolex.txt'
        else:
            raise FileNotFoundError('No EmoLex file found')
    emolex = defaultdict(dict) # Like Counter(), defaultdict eases dictionary creation
    with open(filepath, 'r') as f:
    # emolex file format is: word emotion value
        for line in f:
            word, emotion, value = line.strip().split()
            emolex[word][emotion] = int(value)
    return emolex

# Get EmoLex data. Make sure you set the right file path above.
emolex = read_emolex(emolex_file)

In [8]:
# Get EmoLex df. Make sure you set the right file path above.
emolex = read_emolex(emolex_file)

In [9]:
def getSentScore(sentence_dicts, df, index):
    
    bookdict = dict({'anger': 0 , 'anticipation': 0,'disgust': 0,'fear': 0,'joy': 0,'negative': 0,'positive': 0,'sadness': 0,'surprise': 0, 'trust': 0})
    
    for sentence_dict in sentence_dicts:
        for emotion in sentence_dict:
            bookdict[emotion] += sentence_dict[emotion]
    
    for emotion in bookdict.keys():
        bookdict[emotion] /= len(sentence_dicts)
        df.at[index, emotion] = bookdict[emotion]
    
    return None  

In [16]:
def tokenize_text(text, stops=[]):
    sentences = []
    for sent in sent_tokenize(text.lower()):
        sentences.append([word for word in word_tokenize(sent) if word not in stops])
        
    return sentences

In [17]:
# sentence_sentiment_score from INFO 3350 problem set code
def sentence_sentiment_score(toks, lexicon = emolex):
    print("input sentences: ", toks)
    total = 0
    emo_dict = defaultdict(lambda: 0)
    
    emotions = ['anger', 'anticipation','disgust','fear','joy','negative','positive','sadness','surprise', 'trust']
    
    
    for word in toks:
        print("for ====", word)
        total += 1
        print(lexicon[word])
        for emotion in emotions:
            try:
                emo_dict[emotion] += lexicon[word][emotion]
            except:
                continue
    
    for emotion in emotions:
        if total > 0:
            emo_dict[emotion] /= total
        
    return emo_dict

In [18]:
#### Adding sentiment score columns
size = len(df)

df['anger'] = np.zeros(size)
df['anticipation'] = np.zeros(size)
df['disgust'] = np.zeros(size)
df['fear'] = np.zeros(size)
df['joy'] = np.zeros(size)
df['negative'] = np.zeros(size)
df['positive'] = np.zeros(size)
df['sadness'] = np.zeros(size)
df['surprise'] = np.zeros(size)
df['trust'] = np.zeros(size)

In [19]:
len(df)

1

In [14]:
%%time

for index, text in enumerate(df['risk']):
    sentence_dicts = []
    for sentence in tokenize_text(text, stops=stoplist):
        # print(sentence)
        sentence_dicts.append(sentence_sentiment_score(sentence))
        # print(sentence_dicts)
    # print(len(sentence_dicts))
    # print(index)
    getSentScore(sentence_dicts, df, index)

abandon
++++ [['abandon']]
input sentences:  ['abandon']
for ==== abandon
{'anger': 0, 'anticipation': 0, 'disgust': 0, 'fear': 1, 'joy': 0, 'negative': 1, 'positive': 0, 'sadness': 1, 'surprise': 0, 'trust': 0}
CPU times: user 7.92 ms, sys: 2.65 ms, total: 10.6 ms
Wall time: 11 ms


In [15]:
df

,cik,symbol,company,fiscal_year,risk,url,price,anger,anticipation,disgust,fear,joy,negative,positive,sadness,surprise,trust
0,123,test,apple,2018,abandon,none,100,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0
